In [8]:
import os.path
import PIL
import os
import numpy as np
from PIL import Image
from numpy import asarray

constellations = []
PATH_train = r'C:\Users\leleg\Constellation\Train'
PATH_test  = r'C:\Users\leleg\Constellation\Test'
address_train = []
address_test = []
for path,directory,file in os.walk(r'C:\Users\leleg\Constellation\Train'):
    if(path == PATH_train):
        constellations = directory
    else :
        address_train.append(path)
for path,directory,file in os.walk(r'C:\Users\leleg\Constellation\Test'):
    if(path!=PATH_test):
        address_test.append(path)
DATA_train = []
for i in range(len(constellations)):
    for path,directory,file in os.walk(address_train[i]):
               for f in file:
                   some = os.path.join(path, f)
                   DATA_train.append((some,i))
                
DATA_test = []
for i in range(len(constellations)):
    for path,directory,file in os.walk(address_test[i]):
               for f in file:
                   some = os.path.join(path, f)
                   DATA_test.append((some,i))

X_train =[]
y_train = []
for t in DATA_train:
    img = Image.open(t[0])
    img = asarray(img)
    img = np.transpose(img,(2,1,0))
    X_train.append(img)
    y_train.append(t[1])
X_test =[]
y_test = []
for t in DATA_test:
    img = Image.open(t[0])
    img = asarray(img)
    img = np.transpose(img,(2,1,0))
    X_test.append(img)
    y_test.append(t[1])
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test  = np.array(X_test)
y_test  = np.array(y_test)
print(X_train.shape , y_train.shape , X_test.shape, y_test.shape)
    



(133, 3, 75, 75) (133,) (57, 3, 75, 75) (57,)


In [11]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from collections import Counter
class somecoolname:
    def __init__(self,X,Y,k):
        self.X1 = X
        self.Y = Y
        self.k = k
        print(self.X1.shape)
        area = self.X1.shape[2]*self.X1.shape[3]*self.X1.shape[1]
        length = self.Y.shape[0]
        self.X = np.zeros((length,area))
        for i in range(length):
            self.X[i] = self.X1[i].flatten()
    
        self.X = self.prepro(self.X)
 
        
    def prepro(self,X):
        scaler = MinMaxScaler()
        self.scaler = scaler
        return scaler.fit_transform(X)
    def rotate(self,X):
        return np.rot90(X,k = 1 , axes =(1,2))
    def predict(self,test_im):
        distances = []
        for i in range(self.Y.shape[0]):
            norm = 1e10
            for j in range(4):
                test_vector = test_im.flatten()
                test_vector = np.reshape(test_vector ,(1,test_vector.shape[0]))
                test_vector = self.scaler.transform(test_vector)
                curr = np.linalg.norm(test_vector - self.X[i])
                norm = min(norm,curr)
                test_im = self.rotate(test_im)
            distances.append(norm)
        distances = np.array(distances)
        indexes = np.argsort(distances)[:self.k]
        
        label = []
        for i in range(self.k):
            label.append(self.Y[indexes[i]])
        
        label = (np.array(label))
        return np.bincount(label).argmax()
    def checkacc(self,x_test_l,y_test_l=None):
        if y_test_l is not None:
            predictions = []
            for i in range(y_test_l.shape[0]):
                y_test = y_test_l[i]
                y_pred = self.predict(x_test_l[i])
                predictions.append(y_pred)
            predictions = np.array(predictions)
            #predictions = predictions.reshape((y_test_l.shape[0],1))
            accuracy = np.mean(predictions == y_test_l)*100
            print(accuracy)
            return predictions
        else :
            predictions = []
            for i in range(x_test_l.shape[0]):
                y_pred = self.predict(x_test_l[i])
                predictions.append(y_pred)
            predictions = np.array(predictions)
            predictions = predictions.reshape((x_test_l.shape[0],1)).T
            return predictions
        
    

In [12]:
obj = somecoolname(X_train,y_train,1)

(133, 3, 75, 75)


In [13]:
predictions = obj.checkacc(X_test,y_test)

61.40350877192983


In [14]:
print(predictions)

[ 9  5  0  1  1  1  2  2  2  3  3 10  4  3  4  5  5  5 11 16  5  3 11  7
  7  5 10 16  9  3 10 10  9  3 11 11 12 12 12 13 13 13 14 14 14 15  3 15
 10 10 16 11 10  3 18 18 18]


The predictions that we get are the numbers according to which we had encoded our constellation names initially. The numbering has been done lexicographically. We get an accuracy of 61.40% or 35/57 examples have been classified correctly based on their nearest neighbour. Since the test set consists of 3 images of each constellation, which we have used in the same order without shuffling , the predictions give us the labels predicted by the model.

